In [1]:
from __future__ import print_function
from mosaicml import *
from mosaicml.constants import MLModelFlavours
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
import numpy

Using TensorFlow backend.
/home/pcadmin/PyCharmProjects1/mosaic-ai-client/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pcadmin/PyCharmProjects1/mosaic-ai-client/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pcadmin/PyCharmProjects1/mosaic-ai-client/venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np

In [2]:
if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("NaiveBayesExample")\
        .getOrCreate()

    # $example on$
    # Load training data
    data = spark.read.format("libsvm") \
        .load("sample_libsvm_data.txt")
    
#     Split the data into train and test
    splits = data.randomSplit([0.6, 0.4], 1234)
    train = splits[0]
    test = splits[1]
#     create the trainer and set its parameters
    nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

In [6]:
    # train the model
    model = nb.fit(train)

    # select example rows to display.
    
    predictions = model.transform(test)
    predictions.show()

    # compute accuracy on the test set
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                                  metricName="accuracy")
    accuracy = evaluator.evaluate(predictions)
    print("Test set accuracy = " + str(accuracy))

+-----+--------------------+--------------------+-----------+----------+
|label|            features|       rawPrediction|probability|prediction|
+-----+--------------------+--------------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|[-174115.98587057...|  [1.0,0.0]|       0.0|
|  0.0|(692,[98,99,100,1...|[-178402.52307196...|  [1.0,0.0]|       0.0|
|  0.0|(692,[100,101,102...|[-100905.88974016...|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[-244784.29791241...|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[-196900.88506109...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-238164.45338794...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-184206.87833381...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-214174.52863813...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-182844.62193963...|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|[-246557.10990301...|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|[-208282.08496711...|  

In [7]:
from werkzeug.datastructures import FileStorage

In [8]:
@scoring_func
def score(model, request):
    file= FileStorage(request.files['file1'])
    file.save("f1.txt")
    spark_new = SparkSession.builder.appName("NaiveBayesExample").getOrCreate()
    data = spark_new.read.format("libsvm") \
        .load("f1.txt")
#   Split the data into train and test
    splits = data.randomSplit([0.6, 0.4], 1234)
    train = splits[0]
    test = splits[1]
    
    predictions = model.transform(test)
    print("values1", predictions)
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                                  metricName="accuracy")
    print(predictions.show())
    accuracy = evaluator.evaluate(predictions)
    print("Test set accuracy = " + str(accuracy))
    return accuracy

In [11]:
register_model(model, score, "sample_libsvm_pyspark14", "pyspark model", MLModelFlavours.pyspark)

{'created_by': 'vidhi.shah',
 'created_on': '2019-09-20T11:30:43+00:00',
 'deployments': [],
 'description': 'pyspark model',
 'flavour': 'pyspark',
 'id': '9a721e1f-db11-493d-8fbf-e0143cccef41',
 'init_script': '',
 'last_modified_by': 'vidhi.shah',
 'last_modified_on': None,
 'name': 'sample_libsvm_pyspark14',
 'project_id': '1',
 'resource': {'cpu': '100m',
  'gpu': '',
  'id': '0812875f-7fe1-4624-b696-42809e7dce92',
  'name': 'default',
  'ram': '200MB',
  'replica': '1'},
 'versions': [{'created_by': 'vidhi.shah',
   'created_on': '2019-09-20T11:30:43+00:00',
   'deployments': [],
   'description': None,
   'id': '7c661542-cc33-4047-9a53-801c1562616f',
   'last_modified_by': 'vidhi.shah',
   'last_modified_on': '2019-09-20T11:30:44+00:00',
   'ml_model_id': '9a721e1f-db11-493d-8fbf-e0143cccef41',
   'object_url': '9a721e1f-db11-493d-8fbf-e0143cccef41/7c661542-cc33-4047-9a53-801c1562616f/ml_model.tar.gz',
   'profiling': [],
   'schema': None}]}

In [22]:
load_model("59007214-e480-4651-813b-c8e0d48dabd8", "eb57def4-db07-40a2-87d5-288c54d9907c")

(NaiveBayes_476aba42749a8445d6a5,
 <mosaic_utils.ai.decorators.scoring_func at 0x7f0235fe4110>)

In [9]:
import requests
from werkzeug.datastructures import FileStorage
request = requests.Request()
request.files= {"file1" : open('sample_libsvm_data.txt','rb')}
score(model,request)

values1 DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]
+-----+--------------------+--------------------+-----------+----------+
|label|            features|       rawPrediction|probability|prediction|
+-----+--------------------+--------------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|[-174115.98587057...|  [1.0,0.0]|       0.0|
|  0.0|(692,[98,99,100,1...|[-178402.52307196...|  [1.0,0.0]|       0.0|
|  0.0|(692,[100,101,102...|[-100905.88974016...|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[-244784.29791241...|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[-196900.88506109...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-238164.45338794...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-184206.87833381...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-214174.52863813...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-182844.62193963...|  [1.0,0.0]|       0.0|
|  0.0|(6

1.0